# Create an Index Notebook for all of the Notebooks in a folder

This will create an Index Notebook for all of the notebooks in a folder. It will provide a block per notebook which when run will copy the notebook to a working folder, rename it with the date and user appended and open it ready for use.

Perfect for a set of run books.

It requires the ADSNotebook PowerShell module `Install-Module AdsNotebook -Scope CurrentUser`

To use this Notebook, you wil need to alter some of the variables

`$NotebookDirectory` This is the directory holding the notebooks  
`$targetfilename`  This is the name of the Index Notebook
`$outputLocation` This is the directory that the generated notebook will be placed



In [1]:
$NotebookDirectory = "C:\Users\mrrob\OneDrive\Documents\GitHub\Presentations\2020\SQL Saturday Edinburgh\Notebooks"
$targetfilename = "ADSNotebookIndex.ipynb" 
$outputLocation = 'C:\temp\Work'
$NotebookPath = "$($NotebookDirectory)\$targetfilename"

$IntroCellText = "# Notebook Index 
Click on the code below the workbook you require to generate your custom version to save results in"
$IntroCell = New-ADSWorkBookCell -Text $IntroCellText -Type Text

#check output path exists
Write-Output "Starting"
Write-Output "checking Directory $($outputLocation) exists"
If (!(Test-path $outputLocation)) {
    Write-Output "creating Directory $($outputLocation)"
    New-Item -Path  $outputLocation  -ItemType Directory   
} else {
    Write-Output "Directory $($outputLocation) exists"
}

#check ths files available and build the index
Write-Output "Looping through files in  $($NotebookDirectory) and creating the cells"
$WorkBookCells = foreach ($file in (Get-ChildItem $NotebookDirectory -Recurse -Exclude $targetfilename)) {
    # In case there are subfolders which don't have notebooks in them
    If ($file.PSIsContainer -ieq $true -and ((Get-ChildItem $file.FullName -Recurse -Include *ipynb).Count -ne 0)) {                            
        New-ADSWorkBookCell -Type Text -Text "---
---
## <u>Notebook Section:  **$($file.basename) </u>**"   
    }
    Elseif ($file.Extension -eq '.ipynb') {
        $NotebookContent = Get-Content $file.FullName |ConvertFrom-Json
$FirstCell = $NotebookContent.Cells[0].Source -replace '#','##'|Out-String
$Text = "---
---
## Notebook Name: **$($file.basename)**
$FirstCell"
New-ADSWorkBookCell -Type Text -Text $Text       
        $CellText = "# To run a copy of the Notebook $($file.basename), click the run button.
`$user = `$env:USERNAME.Replace('.','')
`$time = get-date -Format 'yyyyMMddHHmmssms'
`$destination = '{3}\{0}_' + `$time + '_'+ `$user + '{1}'
copy-item -path '{2}' -destination `$destination
# In case there is media or script files as well. Copy Those also
Get-ChildItem '{4}' -Exclude *ipynb | Copy-Item -Destination '{3}' -Recurse -Force
azuredatastudio.cmd `$destination
" -f $file.BaseName, $file.Extension, $file.FullName, $outputLocation , $file.DirectoryName
        New-ADSWorkBookCell -Type code -Text $celltext -Collapse
    }
}
Write-Output "Creating the notebook at $($NotebookPath)"
New-ADSWorkBook -path $NotebookPath -Cells $IntroCell, $WorkBookCells -Type PowerShell
Write-Output "Completed"

Starting
checking Directory C:\temp\Work exists
Directory C:\temp\Work exists
Looping through files in  C:\Users\mrrob\OneDrive\Documents\GitHub\Presentations\2020\SQL Saturday Edinburgh\Notebooks and creating the cells
Creating the notebook at C:\Users\mrrob\OneDrive\Documents\GitHub\Presentations\2020\SQL Saturday Edinburgh\Notebooks\ADSNotebookIndex.ipynb
Completed


In [2]:
pwd


Path          
----          
C:\Users\mrrob


